In [ ]:
import re
import string
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import model_selection, naive_bayes, svm
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report as creport


from multiprocessing import Pool

In [ ]:
train_data = pd.read_csv('train_data.csv')
train_data.head()

In [ ]:
dev_data = pd.read_csv('dev_data.csv')
dev_data.head()

In [ ]:
print("Train data shape:{} \nDev data shape: {}".format(train_data.shape,dev_data.shape))


In [ ]:
train_data.isnull().sum()

In [ ]:
dev_data.isnull().sum()

In [ ]:
train_data['Hate'].value_counts()

In [ ]:
# Data to plot
labels = 'NOT_HS', 'HS'
sizes = [train_data['Hate'].value_counts()[0], train_data['Hate'].value_counts()[1]]
colors = ['lightskyblue', 'yellow']
explode = (0,0.2)  # explode 1st slice

# Plot
plt.pie(sizes, explode=explode, labels=labels, colors=colors,
autopct='%1.1f%%', shadow=True, startangle=140)

plt.axis('equal')
plt.title("Train Data")
plt.show()

In [ ]:
dev_data['Hate'].value_counts()

In [ ]:
# Data to plot
labels = 'NOT_HS', 'HS'
sizes = [dev_data['Hate'].value_counts()[0], dev_data['Hate'].value_counts()[1]]
colors = ['lightskyblue', 'yellow']
explode = (0,0.2)  # explode 1st slice

# Plot
plt.pie(sizes, explode=explode, labels=labels, colors=colors,
autopct='%1.1f%%', shadow=True, startangle=140)

plt.axis('equal')
plt.title("Dev Data")
plt.show()

# Data Cleaning

In [ ]:
def clean(df):
    df = remove_diacritics(df)
    df = normalize_arabic(df)
    df = remove_punctuations(df)
    df = remove_repeating_char(df)
    df= remove_english_word_and_numbers(df)
    df=clean_space(df)
    return df

In [ ]:
arabic_punctuations = '''`÷« »×؛<>٩٨'٧٦٥٤٣٢١٠_()↗*•&^%][ـ،/:"؟.,'{}⋮≈~¦+|٪!”…“–ـ/[]%=#*+\\•~@£·_{}©^®`→°€™›♥←×§″′Â█à…“★”–●â►−¢¬░¶↑±▾	═¦║―¥▓—‹─▒：⊕▼▪†■’▀¨▄♫☆é¯♦¤▲è¸Ã⋅‘∞∙）↓、│（»，♪╩╚³・╦╣╔╗▬❤ïØ¹≤‡₹´'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)


In [ ]:
def remove_diacritics(df):
    df['Tweet'] = df['Tweet'].apply(lambda x: _remove_diacritics(x))
    return df
def _remove_diacritics(x):
    x = str(x)
    x = re.sub(arabic_diacritics, '', x)
    return x

def normalize_arabic(df):
    df['Tweet'] = df['Tweet'].apply(lambda x: _normalize_arabic(x))
    return df
def _normalize_arabic(x):
    x = str(x)
    # added space around puncts after replace
    x = re.sub("[إأآا]", "ا", x)
    x = re.sub("ى", "ي", x)
    x = re.sub("ؤ", "ء", x)
    x = re.sub("ئ", "ء", x)
    x = re.sub("ة", "ه", x)
    x = re.sub("گ", "ك", x)
    return x

def remove_punctuations(df):
    df['Tweet'] = df['Tweet'].apply(lambda x: _remove_punctuations(x))
    return df
def _remove_punctuations(x):
    x = str(x)
    #translator = str.maketrans(' ', ' ', punctuations_list)
    translator = str.maketrans(punctuations_list, ' '*len(punctuations_list))
    return x.translate(translator)

def remove_repeating_char(df):
    df['Tweet'] = df['Tweet'].apply(lambda x: _remove_repeating_char(x))
    return df
def _remove_repeating_char(x):
    x = str(x)
    return re.sub(r'(.)\1+', r'\1', x)

def remove_english_word_and_numbers(df):
    df['Tweet'] = df['Tweet'].apply(lambda x: _remove_english_word_and_numbers(x))
    return df
def _remove_english_word_and_numbers(x):
    x = str(x)
    return re.sub(r'[a-zA-Z0-9]+', '', x)

def clean_space(df):
    compiled_re = re.compile(r"\s+")
    df['Tweet'] = df["Tweet"].apply(lambda x: _clean_space(x, compiled_re))
    return df
def _clean_space(x, compiled_re):
    return compiled_re.sub(" ", x)


In [ ]:
num_cores = 2
def df_parallelize_run(df, func, num_cores=2):
    df_split = np.array_split(df, num_cores)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
train_data_cleaned = df_parallelize_run(train_data, clean)
#train_data_cleaned.to_csv("train_data_cleaned.csv", index=False)    #save the train_data_cleaned dataframe to csv
dev_data_cleaned = df_parallelize_run(dev_data, clean)
#dev_data_cleaned.to_csv("dev_data_cleaned.csv", index=False)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [ ]:
dataset = pd.read_csv('/content/drive/My Drive/OSACT4/train_data_cleaned.csv')

In [ ]:
x=dataset.iloc[:,0]
y=dataset.iloc[:,2]
X=x.to_dict()

X=[]
for d in range(len(x)):
    b=x[d]
    X.append(b) 

In [ ]:
count_vect=CountVectorizer()
X_train_counts=count_vect.fit_transform(X)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf= X_train_tfidf.toarray()

In [ ]:
print(X_train_tfidf.shape)

## SVM Classifier

In [ ]:
from sklearn.svm import SVC
clf= SVC(random_state = 0)
clf.fit(X_train_tfidf, y)
clf.score(X_train_tfidf, y)

In [ ]:
dev_data = pd.read_csv('/content/drive/My Drive/OSACT4/dev_data_cleaned.csv')


In [ ]:
x_dev=dev_data.iloc[:,0]
y_dev=dev_data.iloc[:,2]
#X=x_dev.to_dict()

X=[]
for d in range(len(x)):
    b=x[d].lower()
    X.append(b)  

In [ ]:
X_dev_tfidf=count_vect.transform(x_dev)


In [ ]:
y_pred=clf.predict(X_dev_tfidf)
print(creport(y_dev, y_pred,target_names=['HS', 'NOT_HS'],digits=4))


## Random Forest Classifier


In [ ]:
from sklearn import model_selection, naive_bayes, svm, ensemble, tree
from xgboost import XGBClassifier


In [ ]:
RF= ensemble.RandomForestClassifier()
RF.fit(X_train_tfidf, y)
RF.score(X_train_tfidf, y)

In [ ]:
x_dev=dev_data.iloc[:,0]
y_dev=dev_data.iloc[:,2]
X=x_dev.to_dict()

X=[]
for d in range(len(x)):
    b=x[d].lower()
    X.append(b)

X_dev_tfidff=count_vect.transform(x_dev)
y_pred=RF.predict(X_dev_tfidf)

In [ ]:
print(creport(y_dev, y_pred,target_names=['HS', 'NOT_HS'] ,digits=4))


## Decision Tree Classifier

In [ ]:
DTC= tree.DecisionTreeClassifier()
DTC.fit(X_train_tfidf, y)
DTC.score(X_train_tfidf, y)

In [ ]:
x_dev=dev_data.iloc[:,0]
y_dev=dev_data.iloc[:,2]
X=x_dev.to_dict()

X=[]
for d in range(len(x)):
    b=x[d].lower()
    X.append(b)

X_dev_tfidf=count_vect.transform(x_dev)
y_pred=DTC.predict(X_dev_tfidf)

In [ ]:
print(creport(y_dev, y_pred,target_names=['HS', 'NOT_HS'],digits=4))


## Gradient Boosting Classifier

In [ ]:
GBC= ensemble.GradientBoostingClassifier()
GBC.fit(X_train_tfidf, y)
GBC.score(X_train_tfidf, y)

In [ ]:
x_dev=dev_data.iloc[:,0]
y_dev=dev_data.iloc[:,2]
X=x_dev.to_dict()

X=[]
for d in range(len(x)):
    b=x[d].lower()
    X.append(b)

X_dev_tfidf=count_vect.transform(x_dev)
y_pred=GBC.predict(X_dev_tfidf)

In [ ]:
print(creport(y_dev, y_pred,target_names=['HS', 'NOT_HS'],digits=4))


## Logistic Regression Classifier

In [ ]:
from sklearn import linear_model

In [ ]:
LR= linear_model.LogisticRegression()
LR.fit(X_train_tfidf, y)
LR.score(X_train_tfidf, y)

In [ ]:
x_dev=dev_data.iloc[:,0]
y_dev=dev_data.iloc[:,2]
X=x_dev.to_dict()

X=[]
for d in range(len(x)):
    b=x[d].lower()
    X.append(b)

In [ ]:
X_dev_tfidf=count_vect.transform(x_dev)


In [ ]:
y_pred=LR.predict(X_dev_tfidf)


In [ ]:
print(creport(y_dev, y_pred,target_names=['HS', 'NOT_HS'],digits=4))
